In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Daywise variable and slope
#Cluster your wise
#Day wise date structure do to do
#Attributes of a timeseries 
#How we are picked compound to other countries
#Lockdown impact on graphs 
#Day wise story time graphs of countries
#Predictions of countries
#Predictions of the countries compared to actual

In [ ]:
import pandas
import numpy as np
from sklearn import preprocessing, cross_decomposition,model_selection,metrics
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
%matplotlib inline
import dateutil
from tqdm import tqdm
from sklearn import linear_model
import datetime
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from ipwidgets import widgets

In [ ]:
!Pip install xgboost

In [ ]:
!Pip install plotly

In [ ]:
!Pip install plotly

In [1]:
import pandas as pd
confirmedGlobal=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')

In [2]:
confirmedGlobal.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,53584,53584,53775,53831,53938,53984,54062,54141,54278,54403
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,64627,65334,65994,66635,67216,67690,67982,68568,69238,69916
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,102641,102860,103127,103381,103611,103833,104092,104341,104606,104852
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,8682,8818,8868,8946,9038,9083,9083,9194,9308,9379
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,18343,18425,18613,18679,18765,18875,18926,19011,19093,19177


In [3]:
confirmedGlobal[confirmedGlobal.columns[-1]].sum()

97492076

In [5]:
barPlotData=confirmedGlobal[['Country/Region',confirmedGlobal.columns[-1]]].groupby('Country/Region').sum().sort_values(by=['1/20/22'], ascending=False)

KeyError: '1/20/22'

In [6]:
barPlotData=confirmedGlobal[['Country/Region',confirmedGlobal.columns[-1]]].groupby('Country/Region').sum()
barPlotData=barPlotData.reset_index()
barPlotData.columns=['Country/Region','values']
barPlotData=barPlotData.sort_values(by='values',ascending=False)
barPlotVals=barPlotData['values'].values.tolist()
countryNames=barPlotData['Country/Region'].values.tolist()

In [7]:
df3=pd.read_json('https://cdn.jsdelivr.net/gh/highcharts/highcharts@v7.0.0/samples/data/world-population-density.json')

In [9]:
df3.head(2)

,code3,name,value,code
0,ABW,Aruba,582.34,AW
1,AFG,Afghanistan,53.08,AF


In [10]:
df3[df3['name']=='China']

,code3,name,value,code
35,CHN,China,146.85,CN


In [20]:
dataForMap=[]
for i in countryNames:
    try:
        tempdf=df3[df3['name']==i]
        temp={}
        temp["code3"]=list(tempdf['code3'].values)[0]
        temp["name"]=i
        temp["value"]=barPlotData[barPlotData['Country/Region']==i]['values'].sum()
        temp["code"]=list(tempdf['code'].values)[0]
        dataForMap.append(temp)
    except:
        pass

In [21]:
dataForMap

[{'code3': 'IND', 'name': 'India', 'value': 10625428, 'code': 'IN'},
 {'code3': 'BRA', 'name': 'Brazil', 'value': 8697368, 'code': 'BR'},
 {'code3': 'GBR', 'name': 'United Kingdom', 'value': 3553773, 'code': 'GB'},
 {'code3': 'FRA', 'name': 'France', 'value': 3046371, 'code': 'FR'},
 {'code3': 'ESP', 'name': 'Spain', 'value': 2456675, 'code': 'ES'},
 {'code3': 'ITA', 'name': 'Italy', 'value': 2428221, 'code': 'IT'},
 {'code3': 'TUR', 'name': 'Turkey', 'value': 2412505, 'code': 'TR'},
 {'code3': 'DEU', 'name': 'Germany', 'value': 2108895, 'code': 'DE'},
 {'code3': 'COL', 'name': 'Colombia', 'value': 1972345, 'code': 'CO'},
 {'code3': 'ARG', 'name': 'Argentina', 'value': 1843077, 'code': 'AR'},
 {'code3': 'MEX', 'name': 'Mexico', 'value': 1688944, 'code': 'MX'},
 {'code3': 'POL', 'name': 'Poland', 'value': 1457755, 'code': 'PL'},
 {'code3': 'ZAF', 'name': 'South Africa', 'value': 1380807, 'code': 'ZA'},
 {'code3': 'UKR', 'name': 'Ukraine', 'value': 1216780, 'code': 'UA'},
 {'code3': 'PER

In [22]:
countryDataSpe=pd.DataFrame(confirmedGlobal[confirmedGlobal['Country/Region']=='China'][confirmedGlobal.columns[4:-1]].sum()).reset_index()
countryDataSpe.columns=['country','values']
countryDataSpe['lagVal']=countryDataSpe['values'].shift(1).fillna(0)
countryDataSpe['incrementVal']=countryDataSpe['values']-countryDataSpe['lagVal']
countryDataSpe['rollingMean']=countryDataSpe['incrementVal'].rolling(window=4).mean()
countryDataSpe=countryDataSpe.fillna(0)
datasetsForLine=[{'label':'Daily Cumulated Data','data':countryDataSpe['values'].values.tolist(),'borderColor':'#03a9fc','backgroundColor':'#03a9fc','fill':'false'},
                    {'label':'Rolling Mean 4 days','data':countryDataSpe['rollingMean'].values.tolist(),'borderColor':'#fc5203','backgroundColor':'#fc5203','fill':'false'}]

In [23]:
datasetsForLine

[{'label': 'Daily Cumulated Data',
  'data': [548,
   643,
   920,
   1406,
   2075,
   2877,
   5509,
   6087,
   8141,
   9802,
   11891,
   16630,
   19716,
   23707,
   27440,
   30587,
   34110,
   36814,
   39829,
   42354,
   44386,
   44759,
   59895,
   66358,
   68413,
   70513,
   72434,
   74211,
   74619,
   75077,
   75550,
   77001,
   77022,
   77241,
   77754,
   78166,
   78600,
   78928,
   79356,
   79932,
   80136,
   80261,
   80386,
   80537,
   80690,
   80770,
   80823,
   80860,
   80887,
   80921,
   80932,
   80945,
   80977,
   81003,
   81033,
   81058,
   81102,
   81156,
   81250,
   81305,
   81435,
   81498,
   81591,
   81661,
   81782,
   81897,
   81999,
   82122,
   82198,
   82279,
   82361,
   82432,
   82511,
   82543,
   82602,
   82665,
   82718,
   82809,
   82883,
   82941,
   83014,
   83134,
   83213,
   83306,
   83356,
   83403,
   83760,
   83787,
   83805,
   83817,
   83853,
   83868,
   83884,
   83899,
   83909,
   83912,
   83918,
